In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
import folium
import matplotlib.pyplot as plt
import os

from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import r2_score

/opt/conda/lib/python3.7/site-packages/geopandas/_compat.py:115: UserWarning: The Shapely GEOS version (3.9.1-CAPI-1.14.2) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  shapely_geos_version, geos_capi_version_string


# İl ve İlçeleri İstasyonlarla Eşleştirme

In [2]:
il_shp = gpd.read_file("../input/2204-d-l-ve-l-e-veri-setini-d-zenleme/Il_Sinirlari_Polygon.shp")
il_shp = il_shp.to_crs(epsg=4326)
il_shp['centroid'] = il_shp.centroid
il_shp['centroid'] = il_shp['centroid'].to_crs(epsg=4326)
il_shp.head()

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  This is separate from the ipykernel package so we can avoid doing imports until


,il,geometry,centroid
0,Nigde,"POLYGON ((35.04489 37.98317, 35.03833 37.98151...",POINT (34.54969 37.98591)
1,Konya,"POLYGON ((34.41157 37.35142, 34.39887 37.34340...",POINT (32.81806 37.94680)
2,Mersin,"POLYGON ((34.44162 37.32155, 34.44804 37.32087...",POINT (34.45290 37.29713)
3,Karaman,"POLYGON ((34.28436 37.22108, 34.27774 37.21775...",POINT (33.47320 37.26002)
4,Mersin,"POLYGON ((34.14111 37.19009, 34.13561 37.19010...",POINT (34.02366 37.10653)


In [3]:
ilce_shp = gpd.read_file("../input/2204-d-l-ve-l-e-veri-setini-d-zenleme/Ilce_Sinirlari_Polygon.shp")
ilce_shp = ilce_shp.to_crs(epsg=4326)
ilce_shp['centroid'] = ilce_shp.centroid
ilce_shp['centroid'] = ilce_shp['centroid'].to_crs(epsg=4326)
ilce_shp.head()

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  This is separate from the ipykernel package so we can avoid doing imports until


,ilce,geometry,centroid
0,Nigde_Merkez,"POLYGON ((35.04489 37.98317, 35.04088 37.98216...",POINT (34.74732 38.10945)
1,Camardi,"POLYGON ((34.86361 37.94323, 34.86529 37.93813...",POINT (34.85741 37.91865)
2,Bor,"POLYGON ((34.85726 37.85531, 34.85031 37.84722...",POINT (34.45304 37.85566)
3,Ulukisla,"POLYGON ((34.62206 37.68925, 34.61587 37.68602...",POINT (34.43086 37.57908)
4,Camliyayla,"POLYGON ((34.50508 37.37900, 34.49775 37.37649...",POINT (34.48395 37.37540)


In [4]:
istasyonlar = gpd.read_file("../input/mgm-meteorolojik-dzenlenmi/mygeodata/Ads-z_Proje-point.shp")
istasyonlar = istasyonlar.drop(columns=["descriptio", "altitudeMo"])
istasyonlar

,Name,geometry
0,Konya_B,POINT (32.47130 37.86870)
1,Beysehir,POINT (31.74630 37.67770)
2,Kulu,POINT (33.06570 39.07880)
3,Eregli,POINT (34.04850 37.52550)
4,Konya_H,POINT (32.57400 37.98370)
5,Hadim,POINT (32.45570 36.98930)
6,Ilgin,POINT (31.89400 38.27630)
7,Cumra,POINT (32.79000 37.56580)
8,Karapinar,POINT (33.52600 37.71631)
9,Seydisehir,POINT (31.85380 37.44960)


In [5]:
m = folium.Map(location= (37.86849369999999, 32.471403), zoom_start=8)
folium.GeoJson(data= il_shp["geometry"]).add_to(m)
for i, p in il_shp.iterrows():
    folium.Marker(location = [p.centroid.xy[1][0], p.centroid.xy[0][0]], popup = p.il, 
                  icon = folium.Icon(color = "%s" % "pink")).add_to(m)
for i,p in istasyonlar.iterrows():
    folium.Marker(location = [p.geometry.xy[1][0], p.geometry.xy[0][0]], popup = p.Name, 
                  icon = folium.Icon(color = "%s" % "green")).add_to(m)
m

Ilgın - Kadınhanı - Saraöynü ve Hadim istasyonları Havza Dışında

Bu yüzden bu istasyonları veriden atalım
Fakat eksik veride kullanabilmemiz için bu verileri de saklayalım

In [6]:
distname = istasyonlar["Name"].tolist()
distname

['Konya_B',
 'Beysehir',
 'Kulu',
 'Eregli',
 'Konya_H',
 'Hadim',
 'Ilgin',
 'Cumra',
 'Karapinar',
 'Seydisehir',
 'Sarayonu',
 'Kadinhani',
 'Aksaray',
 'Karaman',
 'Nigde']

In [7]:
yakin_ist_list = []
coord_ist_list = []
for i in il_shp.T:
    distlist = []
    for j in istasyonlar.T:
        distlist.append(il_shp.iloc[i]["centroid"].distance(istasyonlar.iloc[j]["geometry"]))
        
    minimum = distlist[0]
    for i in distlist:
        if minimum > i:
            minimum = i

    index = distlist.index(minimum)
    yakin_ist_list.append(distname[index])
    coord_ist_list.append(istasyonlar["geometry"][index])
    
il_shp["yakin_ist"] = yakin_ist_list
il_shp["coord_ist"] = coord_ist_list

il_shp.head()

/opt/conda/lib/python3.7/site-packages/pandas/core/dtypes/cast.py:118: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)


,il,geometry,centroid,yakin_ist,coord_ist
0,Nigde,"POLYGON ((35.04489 37.98317, 35.03833 37.98151...",POINT (34.54969 37.98591),Nigde,POINT (34.6795 37.9585)
1,Konya,"POLYGON ((34.41157 37.35142, 34.39887 37.34340...",POINT (32.81806 37.94680),Konya_H,POINT (32.574 37.9837)
2,Mersin,"POLYGON ((34.44162 37.32155, 34.44804 37.32087...",POINT (34.45290 37.29713),Eregli,POINT (34.0485 37.5255)
3,Karaman,"POLYGON ((34.28436 37.22108, 34.27774 37.21775...",POINT (33.47320 37.26002),Karaman,POINT (33.2202 37.1932)
4,Mersin,"POLYGON ((34.14111 37.19009, 34.13561 37.19010...",POINT (34.02366 37.10653),Eregli,POINT (34.0485 37.5255)


In [8]:
m = folium.Map(location= (37.86849369999999, 32.471403), zoom_start=8)
folium.GeoJson(data= il_shp["geometry"]).add_to(m)
for i, p in il_shp.iterrows():
    folium.Marker(location = [p.centroid.xy[1][0], p.centroid.xy[0][0]], popup = "İl: "+p.il+"\nİst: "+p.yakin_ist, 
                  icon = folium.Icon(color = "%s" % "pink")).add_to(m)
for i,p in istasyonlar.iterrows():
    folium.Marker(location = [p.geometry.xy[1][0], p.geometry.xy[0][0]], popup = "İst: "+p.Name, 
                  icon = folium.Icon(color = "%s" % "green")).add_to(m)
m

Tek düzelteceğimiz şey Konya için içerisindeki tüm istasyonların ortalamasını almak olcak. Bunun için KONYA_ORT istasyonu yazacağız.

In [9]:
c_il_shp = il_shp.drop(columns=["geometry", "centroid", "coord_ist"])
c_il_shp

,il,yakin_ist
0,Nigde,Nigde
1,Konya,Konya_H
2,Mersin,Eregli
3,Karaman,Karaman
4,Mersin,Eregli
5,Mersin,Karaman
6,Antalya,Seydisehir
7,Antalya,Seydisehir
8,Antalya,Seydisehir
9,Antalya,Seydisehir


In [10]:
for i,r in c_il_shp.iterrows():
    if r.il == "Mersin":
        r.yakin_ist = "Karaman"
    elif r.il == "Konya":
        r.yakin_ist = "KONYA_ORT"

In [11]:
c_il_shp = c_il_shp.drop_duplicates()
c_il_shp

,il,yakin_ist
0,Nigde,Nigde
1,Konya,KONYA_ORT
2,Mersin,Karaman
3,Karaman,Karaman
6,Antalya,Seydisehir
10,Isparta,Beysehir
11,Ankara,Kulu
13,Aksaray,Aksaray
15,Nevsehir,Nigde


Şimdi ilçeler ile yapalım

In [12]:
yakin_ist_list = []
coord_ist_list = []
for i in ilce_shp.T:
    distlist = []
    for j in istasyonlar.T:
        distlist.append(ilce_shp.iloc[i]["centroid"].distance(istasyonlar.iloc[j]["geometry"]))
        
    minimum = distlist[0]
    for i in distlist:
        if minimum > i:
            minimum = i

    index = distlist.index(minimum)
    yakin_ist_list.append(distname[index])
    coord_ist_list.append(istasyonlar["geometry"][index])
    
ilce_shp["yakin_ist"] = yakin_ist_list
ilce_shp["coord_ist"] = coord_ist_list

ilce_shp.head()

/opt/conda/lib/python3.7/site-packages/pandas/core/dtypes/cast.py:118: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)


,ilce,geometry,centroid,yakin_ist,coord_ist
0,Nigde_Merkez,"POLYGON ((35.04489 37.98317, 35.04088 37.98216...",POINT (34.74732 38.10945),Nigde,POINT (34.6795 37.9585)
1,Camardi,"POLYGON ((34.86361 37.94323, 34.86529 37.93813...",POINT (34.85741 37.91865),Nigde,POINT (34.6795 37.9585)
2,Bor,"POLYGON ((34.85726 37.85531, 34.85031 37.84722...",POINT (34.45304 37.85566),Nigde,POINT (34.6795 37.9585)
3,Ulukisla,"POLYGON ((34.62206 37.68925, 34.61587 37.68602...",POINT (34.43086 37.57908),Eregli,POINT (34.0485 37.5255)
4,Camliyayla,"POLYGON ((34.50508 37.37900, 34.49775 37.37649...",POINT (34.48395 37.37540),Eregli,POINT (34.0485 37.5255)


In [13]:
m = folium.Map(location= (37.86849369999999, 32.471403), zoom_start=8)
folium.GeoJson(data= ilce_shp["geometry"]).add_to(m)
for i, p in ilce_shp.iterrows():
    folium.Marker(location = [p.centroid.xy[1][0], p.centroid.xy[0][0]], popup = "İlçe: "+p.ilce+"\nİst: "+p.yakin_ist, 
                  icon = folium.Icon(color = "%s" % "pink")).add_to(m)
for i,p in istasyonlar.iterrows():
    folium.Marker(location = [p.geometry.xy[1][0], p.geometry.xy[0][0]], popup = "İst: "+p.Name, 
                  icon = folium.Icon(color = "%s" % "green")).add_to(m)
m

In [14]:
c_ilce_shp = ilce_shp.drop(columns=["coord_ist"])
for i,r in c_ilce_shp.iterrows():
    if r.ilce == "Yalvac" or r.ilce == "Ilgin" or r.ilce == "Doganhisar":
        r.yakin_ist = "Beysehir"
    elif r.ilce == "Bozkir":
        r.yakin_ist = "Seydisehir"
    elif r.ilce == "Akoren":
        r.yakin_ist = "Cumra"
    elif r.ilce == "Derbent" or r.ilce == "Kadinhani" or r.ilce == "Sarayonu" or r.ilce == "Karatay" or r.ilce == "Konya_B":
        r.yakin_ist = "Konya_H"
        
c_ilce_shp = c_ilce_shp.drop_duplicates()
        
c_ilce_shp

,ilce,geometry,centroid,yakin_ist
0,Nigde_Merkez,"POLYGON ((35.04489 37.98317, 35.04088 37.98216...",POINT (34.74732 38.10945),Nigde
1,Camardi,"POLYGON ((34.86361 37.94323, 34.86529 37.93813...",POINT (34.85741 37.91865),Nigde
2,Bor,"POLYGON ((34.85726 37.85531, 34.85031 37.84722...",POINT (34.45304 37.85566),Nigde
3,Ulukisla,"POLYGON ((34.62206 37.68925, 34.61587 37.68602...",POINT (34.43086 37.57908),Eregli
4,Camliyayla,"POLYGON ((34.50508 37.37900, 34.49775 37.37649...",POINT (34.48395 37.37540),Eregli
...,...,...,...,...
77,Akoren,"POLYGON ((32.39691 37.30014, 32.39691 37.30013...",POINT (32.36484 37.42057),Hadim
78,Yalihuyuk,"POLYGON ((32.10773 37.36304, 32.10972 37.35945...",POINT (32.07381 37.31112),Seydisehir
79,Meram,"POLYGON ((32.21194 37.45399, 32.20759 37.45533...",POINT (32.29627 37.70609),Konya_B
80,Karatay,"POLYGON ((33.15576 37.70506, 33.14416 37.70148...",POINT (32.98949 37.93189),Cumra


In [15]:
c_il_shp.to_csv("il_istasyon.csv")
c_ilce_shp.to_csv("ilce_istasyon.csv")

# İstasyon Verilerini Tek Bir Dosya Haline Getirme

In [16]:
aks = []
bey = []
cum = []
sey = []
kon = []
kul = []
nig = []
krm = []
krp = []
ere = []

in_dir = os.listdir("../input/mgm-meteorolojik-dzenlenmi")
in_dir.sort()

for i in in_dir:
    if "Aksaray" in i:
        aks.append(i)
    elif "Beysehir" in i:
        bey.append(i)
    elif "Cumra" in i:
        cum.append(i)
    elif "Seydisehir" in i:
        sey.append(i)
    elif "Karaman" in i:
        krm.append(i)
    elif "Konya_H" in i:
        kon.append(i)
    elif "Karapinar" in i:
        krp.append(i)
    elif "Eregli" in i:
        ere.append(i)
    elif "Nigde" in i:
        nig.append(i)
    elif "Kulu" in i:
        kul.append(i)
        
aks

['Aylik_Maks_Sic - Aksaray.csv',
 'Aylik_Max_Yagis - Aksaray.csv',
 'Aylik_Min_Sic - Aksaray.csv',
 'Aylik_Ort_Min_Sic - Aksaray.csv',
 'Aylik_Ort_Sic - Aksaray.csv',
 'Aylik_Toplam_Yagis - Aksaray.csv',
 'Aylk_Ort_Maks_Sic - Aksaray.csv']

In [17]:
kon

['Aylik_Maks_Sic - Konya_H.csv',
 'Aylik_Min_Sic - Konya_H.csv',
 'Aylik_Ort_Min_Sic - Konya_H.csv',
 'Aylik_Ort_Sic - Konya_H.csv',
 'Aylk_Ort_Maks_Sic - Konya_H.csv']

In [18]:
dflist = []

In [19]:
def merging_params(param_lst, il):
    lstoflst = []
    for i in param_lst:
        path = os.path.join("../input/mgm-meteorolojik-dzenlenmi", i)
        df = pd.read_csv(path, sep=",")
        lst = df.drop(columns="Yıl/Ay").values.flatten()
        lstoflst.append(lst)
        in_dir.remove(i)
        
    try:
        out_df = pd.DataFrame({"İstasyon":il, "Yıl":[x for x in range(2000,2022) for y in range(1,13)], 
                               "Ay":[y for x in range(2000,2022) for y in range(1,13)], "max_sic":lstoflst[0],
                               "max_yag":lstoflst[1], "min_sic":lstoflst[2], "ortmin_sic":lstoflst[3], 
                               "ort_sic":lstoflst[4], "top_yag":lstoflst[5], "ortmax_sic":lstoflst[6]})
        
        dflist.append(out_df)
    
        return out_df
    
    except IndexError:
        print("Verilen {} listesinde bazı değerler eksik. Bulunanlar şunlar: ".format(il), param_lst)

In [20]:
aks_df = merging_params(aks, "Aksaray")
bey_df = merging_params(bey, "Beyşehir")
cum_df = merging_params(cum, "Çumra")
sey_df = merging_params(sey, "Seydişehir")
kon_df = merging_params(kon, "Konya")
kul_df = merging_params(kul, "Kulu")
nig_df = merging_params(nig, "Niğde")
krm_df = merging_params(krm, "Karaman")
krp_df = merging_params(krp, "Karapınar")
ere_df = merging_params(ere, "Ereğli")

Verilen Konya listesinde bazı değerler eksik. Bulunanlar şunlar:  ['Aylik_Maks_Sic - Konya_H.csv', 'Aylik_Min_Sic - Konya_H.csv', 'Aylik_Ort_Min_Sic - Konya_H.csv', 'Aylik_Ort_Sic - Konya_H.csv', 'Aylk_Ort_Maks_Sic - Konya_H.csv']


In [21]:
aks_df

,İstasyon,Yıl,Ay,max_sic,max_yag,min_sic,ortmin_sic,ort_sic,top_yag,ortmax_sic
0,Aksaray,2000,1,14.2,NaN,-16.0,-8.1,-3.7,NaN,0.6
1,Aksaray,2000,2,11.9,NaN,-12.0,-5.2,-1.5,NaN,3.1
2,Aksaray,2000,3,23.2,NaN,-9.3,-1.4,4.3,NaN,10.9
3,Aksaray,2000,4,26.6,NaN,-2.0,7.8,13.8,NaN,20.4
4,Aksaray,2000,5,27.0,NaN,4.7,9.6,15.2,NaN,21.0
...,...,...,...,...,...,...,...,...,...,...
259,Aksaray,2021,8,37.6,7.6,14.6,17.1,24.3,16.4,31.3
260,Aksaray,2021,9,33.3,9.3,5.8,12.7,18.4,12.7,24.9
261,Aksaray,2021,10,27.2,0.0,1.8,6.7,13.0,0.0,20.5
262,Aksaray,2021,11,22.0,9.4,-5.1,4.4,9.7,17.7,16.5


Fakat Konya değerlerinden bazıları eksik. Eksik olan değerler toplam yağış ve maksimum yağış. Bu değerleri yakınlardaki bir istasyona göre doldurmamız lazım.

In [22]:
in_dir

['Aylik_Maks_Sic - Hadim.csv',
 'Aylik_Maks_Sic - Ilgin.csv',
 'Aylik_Max_Yagis - Hadim.csv',
 'Aylik_Max_Yagis - Ilgin.csv',
 'Aylik_Max_Yagis - Kadinhani.csv',
 'Aylik_Max_Yagis - Konya_B.csv',
 'Aylik_Max_Yagis - Sarayonu.csv',
 'Aylik_Max_Yagis - Sereflikochisar.csv',
 'Aylik_Min_Sic - Hadim.csv',
 'Aylik_Min_Sic - Ilgin.csv',
 'Aylik_Min_Sic - Konya.csv',
 'Aylik_Min_Sic - Sereflikochisar.csv',
 'Aylik_Ort_Min_Sic - Hadim.csv',
 'Aylik_Ort_Min_Sic - Ilgin.csv',
 'Aylik_Ort_Min_Sic - Konya.csv',
 'Aylik_Ort_Min_Sic - Sereflikochisar.csv',
 'Aylik_Ort_Sic - Hadim.csv',
 'Aylik_Ort_Sic - Ilgin.csv',
 'Aylik_Ort_Sic - Konya.csv',
 'Aylik_Ort_Sic - Sereflikochisar.csv',
 'Aylik_Toplam_Yagis - Hadim.csv',
 'Aylik_Toplam_Yagis - Ilgin.csv',
 'Aylik_Toplam_Yagis - Kadinhani.csv',
 'Aylik_Toplam_Yagis - Konya.csv',
 'Aylik_Toplam_Yagis - Sarayonu.csv',
 'Aylik_Toplam_Yagis - Sereflikochisar.csv',
 'Aylk_Ort_Maks_Sic - Hadim.csv',
 'Aylk_Ort_Maks_Sic - Ilgin.csv',
 'Aylk_Ort_Maks_Sic - Kony

Buradan en yakın istasyon Konya Bölge istasyonu. Bu yüzden bu verileri kullanarak oluşturacağız.

In [23]:
kon

['Aylik_Maks_Sic - Konya_H.csv',
 'Aylik_Min_Sic - Konya_H.csv',
 'Aylik_Ort_Min_Sic - Konya_H.csv',
 'Aylik_Ort_Sic - Konya_H.csv',
 'Aylk_Ort_Maks_Sic - Konya_H.csv']

In [24]:
kon_max_sic = pd.read_csv("../input/mgm-meteorolojik-dzenlenmi/Aylik_Maks_Sic - Konya_H.csv", sep=",")
kon_min_sic = pd.read_csv("../input/mgm-meteorolojik-dzenlenmi/Aylik_Min_Sic - Konya_H.csv", sep=",")
kon_ortmin_sic = pd.read_csv("../input/mgm-meteorolojik-dzenlenmi/Aylik_Ort_Min_Sic - Konya_H.csv", sep=",")
kon_ort_sic = pd.read_csv("../input/mgm-meteorolojik-dzenlenmi/Aylik_Ort_Sic - Konya_H.csv", sep=",")
kon_ortmax_sic = pd.read_csv("../input/mgm-meteorolojik-dzenlenmi/Aylk_Ort_Maks_Sic - Konya_H.csv", sep=",")
kon_top_yag = pd.read_csv("../input/mgm-meteorolojik-dzenlenmi/Aylik_Toplam_Yagis - Konya.csv", sep=",")
in_dir.remove("Aylik_Toplam_Yagis - Konya.csv")
kon_max_yag = pd.read_csv("../input/mgm-meteorolojik-dzenlenmi/Aylik_Max_Yagis - Konya_B.csv", sep=",")
in_dir.remove("Aylik_Max_Yagis - Konya_B.csv")

kon_df = pd.DataFrame({"İstasyon":"Konya", "Yıl":[x for x in range(2000,2022) for y in range(1,13)], 
                        "Ay":[y for x in range(2000,2022) for y in range(1,13)],
                       "max_sic":kon_max_sic.drop(columns="Yıl/Ay").values.flatten(),
                       "max_yag":kon_max_yag.drop(columns="Yıl/Ay").values.flatten(),
                       "min_sic":kon_min_sic.drop(columns="Yıl/Ay").values.flatten(),
                       "ortmin_sic":kon_ortmin_sic.drop(columns="Yıl/Ay").values.flatten(),
                       "ort_sic":kon_ort_sic.drop(columns="Yıl/Ay").values.flatten(),
                       "top_yag":kon_top_yag.drop(columns="Yıl/Ay").values.flatten(),
                       "ortmax_sic":kon_ortmax_sic.drop(columns="Yıl/Ay").values.flatten()})

dflist.append(kon_df)

kon_df

,İstasyon,Yıl,Ay,max_sic,max_yag,min_sic,ortmin_sic,ort_sic,top_yag,ortmax_sic
0,Konya,2000,1,16.2,NaN,-18.2,-0.6,-5.2,NaN,-0.6
1,Konya,2000,2,12.2,NaN,-11.2,2.4,-2.3,NaN,2.4
2,Konya,2000,3,25.2,NaN,-11.8,11.7,3.8,NaN,11.7
3,Konya,2000,4,26.0,NaN,-2.0,19.9,12.7,NaN,19.9
4,Konya,2000,5,26.0,NaN,4.4,21.2,14.6,NaN,21.2
...,...,...,...,...,...,...,...,...,...,...
259,Konya,2021,8,36.3,5.2,13.4,30.6,24.3,5.2,30.6
260,Konya,2021,9,31.8,11.0,2.9,23.9,18.0,33.6,23.9
261,Konya,2021,10,26.0,0.0,1.7,19.1,12.6,0.0,19.1
262,Konya,2021,11,21.0,9.0,-6.2,14.8,8.5,25.6,14.8


In [25]:
aks_df

,İstasyon,Yıl,Ay,max_sic,max_yag,min_sic,ortmin_sic,ort_sic,top_yag,ortmax_sic
0,Aksaray,2000,1,14.2,NaN,-16.0,-8.1,-3.7,NaN,0.6
1,Aksaray,2000,2,11.9,NaN,-12.0,-5.2,-1.5,NaN,3.1
2,Aksaray,2000,3,23.2,NaN,-9.3,-1.4,4.3,NaN,10.9
3,Aksaray,2000,4,26.6,NaN,-2.0,7.8,13.8,NaN,20.4
4,Aksaray,2000,5,27.0,NaN,4.7,9.6,15.2,NaN,21.0
...,...,...,...,...,...,...,...,...,...,...
259,Aksaray,2021,8,37.6,7.6,14.6,17.1,24.3,16.4,31.3
260,Aksaray,2021,9,33.3,9.3,5.8,12.7,18.4,12.7,24.9
261,Aksaray,2021,10,27.2,0.0,1.8,6.7,13.0,0.0,20.5
262,Aksaray,2021,11,22.0,9.4,-5.1,4.4,9.7,17.7,16.5


In [26]:
kon_df

,İstasyon,Yıl,Ay,max_sic,max_yag,min_sic,ortmin_sic,ort_sic,top_yag,ortmax_sic
0,Konya,2000,1,16.2,NaN,-18.2,-0.6,-5.2,NaN,-0.6
1,Konya,2000,2,12.2,NaN,-11.2,2.4,-2.3,NaN,2.4
2,Konya,2000,3,25.2,NaN,-11.8,11.7,3.8,NaN,11.7
3,Konya,2000,4,26.0,NaN,-2.0,19.9,12.7,NaN,19.9
4,Konya,2000,5,26.0,NaN,4.4,21.2,14.6,NaN,21.2
...,...,...,...,...,...,...,...,...,...,...
259,Konya,2021,8,36.3,5.2,13.4,30.6,24.3,5.2,30.6
260,Konya,2021,9,31.8,11.0,2.9,23.9,18.0,33.6,23.9
261,Konya,2021,10,26.0,0.0,1.7,19.1,12.6,0.0,19.1
262,Konya,2021,11,21.0,9.0,-6.2,14.8,8.5,25.6,14.8


Fakat çoğu istasyonda yağış değerleri eksik. Bunlar nereler bakalım.

Ayrıca bu değerleri neyle doldurabiliriz diye csv dosyalarını inceleylim. İncelediğimizde yağışla ilgili en erken ulaşabildiğimiz veriler 2005 yılına ait. Bunlar da sadece Konya Bölge istasyonlarına ait.

In [27]:
"""kon_max_yag = pd.read_csv("../input/mgm-meteorolojik-dzenlenmi/Aylik_Max_Yagis - Konya_B.csv")
kon_top_yag = pd.read_csv("../input/mgm-meteorolojik-dzenlenmi/Aylik_Toplam_Yagis - Konya.csv")

kon_max_yag = pd.DataFrame({"İstasyon":"Konya_Bölge",  "Yıl":[x for x in range(2000,2022) for y in range(1,13)], 
                               "Ay":[y for x in range(2000,2022) for y in range(1,13)],
                            "max_yag":kon_max_yag.drop(columns="Yıl/Ay").values.flatten()})

kon_top_yag = pd.DataFrame({"İstasyon":"Konya_Bölge", "Yıl":[x for x in range(2000,2022) for y in range(1,13)], 
                               "Ay":[y for x in range(2000,2022) for y in range(1,13)],
                            "top_yag":kon_top_yag.drop(columns="Yıl/Ay").values.flatten()})"""

'kon_max_yag = pd.read_csv("../input/mgm-meteorolojik-dzenlenmi/Aylik_Max_Yagis - Konya_B.csv")\nkon_top_yag = pd.read_csv("../input/mgm-meteorolojik-dzenlenmi/Aylik_Toplam_Yagis - Konya.csv")\n\nkon_max_yag = pd.DataFrame({"İstasyon":"Konya_Bölge",  "Yıl":[x for x in range(2000,2022) for y in range(1,13)], \n                               "Ay":[y for x in range(2000,2022) for y in range(1,13)],\n                            "max_yag":kon_max_yag.drop(columns="Yıl/Ay").values.flatten()})\n\nkon_top_yag = pd.DataFrame({"İstasyon":"Konya_Bölge", "Yıl":[x for x in range(2000,2022) for y in range(1,13)], \n                               "Ay":[y for x in range(2000,2022) for y in range(1,13)],\n                            "top_yag":kon_top_yag.drop(columns="Yıl/Ay").values.flatten()})'

In [28]:
"""def nan_doldur(df, param_df, param):
    index=0
    for i in df[param].isna():
        if i == True:
            df[param][index] = param_df[param][index]

        index +=1"""

'def nan_doldur(df, param_df, param):\n    index=0\n    for i in df[param].isna():\n        if i == True:\n            df[param][index] = param_df[param][index]\n\n        index +=1'

In [29]:
"""nan_doldur(aks_df, kon_max_yag, "max_yag")
nan_doldur(bey_df, kon_max_yag, "max_yag")
nan_doldur(cum_df, kon_max_yag, "max_yag")
nan_doldur(sey_df, kon_max_yag, "max_yag")
nan_doldur(krm_df, kon_max_yag, "max_yag")
nan_doldur(nig_df, kon_max_yag, "max_yag")
nan_doldur(krp_df, kon_max_yag, "max_yag")
nan_doldur(ere_df, kon_max_yag, "max_yag")
nan_doldur(kon_df, kon_max_yag, "max_yag")
nan_doldur(kul_df, kon_max_yag, "max_yag")

nan_doldur(aks_df, kon_top_yag, "top_yag")
nan_doldur(bey_df, kon_top_yag, "top_yag")
nan_doldur(cum_df, kon_top_yag, "top_yag")
nan_doldur(sey_df, kon_top_yag, "top_yag")
nan_doldur(krm_df, kon_top_yag, "top_yag")
nan_doldur(nig_df, kon_top_yag, "top_yag")
nan_doldur(krp_df, kon_top_yag, "top_yag")
nan_doldur(ere_df, kon_top_yag, "top_yag")
nan_doldur(kon_df, kon_top_yag, "top_yag")
nan_doldur(kul_df, kon_top_yag, "top_yag")"""

'nan_doldur(aks_df, kon_max_yag, "max_yag")\nnan_doldur(bey_df, kon_max_yag, "max_yag")\nnan_doldur(cum_df, kon_max_yag, "max_yag")\nnan_doldur(sey_df, kon_max_yag, "max_yag")\nnan_doldur(krm_df, kon_max_yag, "max_yag")\nnan_doldur(nig_df, kon_max_yag, "max_yag")\nnan_doldur(krp_df, kon_max_yag, "max_yag")\nnan_doldur(ere_df, kon_max_yag, "max_yag")\nnan_doldur(kon_df, kon_max_yag, "max_yag")\nnan_doldur(kul_df, kon_max_yag, "max_yag")\n\nnan_doldur(aks_df, kon_top_yag, "top_yag")\nnan_doldur(bey_df, kon_top_yag, "top_yag")\nnan_doldur(cum_df, kon_top_yag, "top_yag")\nnan_doldur(sey_df, kon_top_yag, "top_yag")\nnan_doldur(krm_df, kon_top_yag, "top_yag")\nnan_doldur(nig_df, kon_top_yag, "top_yag")\nnan_doldur(krp_df, kon_top_yag, "top_yag")\nnan_doldur(ere_df, kon_top_yag, "top_yag")\nnan_doldur(kon_df, kon_top_yag, "top_yag")\nnan_doldur(kul_df, kon_top_yag, "top_yag")'

In [30]:
"""aks_df.loc[60]"""

'aks_df.loc[60]'

In [31]:
"""kul_df.loc[97]"""

'kul_df.loc[97]'

In [32]:
"""kul_df"""

'kul_df'

Eksik veriler dolduruldu fakat 2005'ten öncesi dolmadı. Bu yüzden 2005'ten önceki verileri silmek zorundayız. Fakat diğer eksik verileri doldurduktan sonra silmemiz gerek.

In [33]:
"""aks_df.isnull().sum()"""

'aks_df.isnull().sum()'

Görüldüğü üzere hala bazı değerlerimiz nan. Bunları komşu istasyonun verisinden alacağız. Fakat bu sefer sadece Konya Bölge istasyonlarından yapmamıza gerek yok. Bütün istasyonları karşılaştıralım.

In [34]:
"""m = folium.Map(location= (37.86849369999999, 32.471403), zoom_start=8)
folium.GeoJson(data= il_shp["geometry"]).add_to(m)
for i,p in istasyonlar.iterrows():
    folium.Marker(location = [p.geometry.xy[1][0], p.geometry.xy[0][0]], popup = "İst: "+p.Name, 
                  icon = folium.Icon(color = "%s" % "green")).add_to(m)
m"""

'm = folium.Map(location= (37.86849369999999, 32.471403), zoom_start=8)\nfolium.GeoJson(data= il_shp["geometry"]).add_to(m)\nfor i,p in istasyonlar.iterrows():\n    folium.Marker(location = [p.geometry.xy[1][0], p.geometry.xy[0][0]], popup = "İst: "+p.Name, \n                  icon = folium.Icon(color = "%s" % "green")).add_to(m)\nm'

In [35]:
"""def nangoster(df):
    ort_sic_nan = list(df.loc[pd.isna(df["ort_sic"])].index) 
    max_sic_nan = list(df.loc[pd.isna(df["max_sic"])].index) 
    min_sic_nan = list(df.loc[pd.isna(df["min_sic"])].index) 
    ortmax_sic_nan = list(df.loc[pd.isna(df["ortmax_sic"])].index) 
    ortmin_sic_nan = list(df.loc[pd.isna(df["ortmin_sic"])].index) 
    #max_yag_nan = list(df.loc[pd.isna(df["max_yag"])].index) 
    #top_yag_nan = list(df.loc[pd.isna(df["top_yag"])].index) 
    #Önceden sildiğimiz için gerek yok
    
    d = {"ort_sic_nan" : ort_sic_nan, "max_sic_nan":max_sic_nan, "min_sic_nan": min_sic_nan,
         "ortmax_sic_nan" : ortmax_sic_nan, "ortmin_sic_nan" : ortmin_sic_nan}
    return d"""

'def nangoster(df):\n    ort_sic_nan = list(df.loc[pd.isna(df["ort_sic"])].index) \n    max_sic_nan = list(df.loc[pd.isna(df["max_sic"])].index) \n    min_sic_nan = list(df.loc[pd.isna(df["min_sic"])].index) \n    ortmax_sic_nan = list(df.loc[pd.isna(df["ortmax_sic"])].index) \n    ortmin_sic_nan = list(df.loc[pd.isna(df["ortmin_sic"])].index) \n    #max_yag_nan = list(df.loc[pd.isna(df["max_yag"])].index) \n    #top_yag_nan = list(df.loc[pd.isna(df["top_yag"])].index) \n    #Önceden sildiğimiz için gerek yok\n    \n    d = {"ort_sic_nan" : ort_sic_nan, "max_sic_nan":max_sic_nan, "min_sic_nan": min_sic_nan,\n         "ortmax_sic_nan" : ortmax_sic_nan, "ortmin_sic_nan" : ortmin_sic_nan}\n    return d'

In [36]:
"""nangoster(aks_df)"""

'nangoster(aks_df)'

In [37]:
"""nig_df.isnull().sum()"""

'nig_df.isnull().sum()'

In [38]:
"""kul_df.isnull().sum()"""

'kul_df.isnull().sum()'

Aksaraydaki kayıp veriyi Kulu istasyonuna ait veride kullanalım.

In [39]:
"""def nanreplace(df, kim_df):
    ind = df.loc[pd.isna(df["ortmax_sic"])].index
    for i in ind:
        df["ortmax_sic"][i] = kim_df["ortmax_sic"][i]
        df["max_sic"][i] = kim_df["max_sic"][i]
        df["ort_sic"][i] = kim_df["ort_sic"][i]
        df["min_sic"][i] = kim_df["min_sic"][i]
        df["ortmin_sic"][i] = kim_df["ortmin_sic"][i]
    
        print(df.loc[i])"""

'def nanreplace(df, kim_df):\n    ind = df.loc[pd.isna(df["ortmax_sic"])].index\n    for i in ind:\n        df["ortmax_sic"][i] = kim_df["ortmax_sic"][i]\n        df["max_sic"][i] = kim_df["max_sic"][i]\n        df["ort_sic"][i] = kim_df["ort_sic"][i]\n        df["min_sic"][i] = kim_df["min_sic"][i]\n        df["ortmin_sic"][i] = kim_df["ortmin_sic"][i]\n    \n        print(df.loc[i])'

In [40]:
"""nanreplace(aks_df, kul_df)"""

'nanreplace(aks_df, kul_df)'

Az önce Niğde'de veri kayıp çıktı. Kaydedelim.

In [41]:
"""ere_df.isnull().sum()"""

'ere_df.isnull().sum()'

In [42]:
"""nanreplace(nig_df, ere_df)"""

'nanreplace(nig_df, ere_df)'

In [43]:
"""bey_df.isnull().sum()"""

'bey_df.isnull().sum()'

In [44]:
#cum_df.isnull().sum()

In [45]:
#krp_df.isnull().sum()

In [46]:
#sey_df.isnull().sum()

In [47]:
#nanreplace(sey_df, bey_df)

In [48]:
#kon_df.isnull().sum()

In [49]:
#nangoster(kon_df)

Konya Bölge verisinden alıcağız. Normal Excel sayfasına bakıyoruz.

In [50]:
#kon_df.loc[pd.isna(kon_df["ortmax_sic"])]

In [51]:
"""kon_df["ortmax_sic"][96] = 1.7
kon_df["max_sic"][96] = 8.4
kon_df["min_sic"][96] = -13.0
kon_df["ortmin_sic"][96] = -5.7

kon_df.loc[96]"""

'kon_df["ortmax_sic"][96] = 1.7\nkon_df["max_sic"][96] = 8.4\nkon_df["min_sic"][96] = -13.0\nkon_df["ortmin_sic"][96] = -5.7\n\nkon_df.loc[96]'

In [52]:
#krp_df.isnull().sum()

In [53]:
#krm_df.isnull().sum()

In [54]:
#nanreplace(krm_df, cum_df)

In [55]:
#krm_df.isnull().sum()

In [56]:
#kon_df.isnull().sum()

Fakat görüldüğü üzere yağış değerlerinin hepsinde 60 nan değeri var. Bunun sebebi elimizde hiçbir istasyona ait elimizde 2000-2004 ve 20005'in 10.ayına kadar veri bulunmaması. 

# Konya İl NDVI için verilerin Ortalamasını Alma

In [57]:
m_ort_sic_lst = []
m_ortmax_sic_lst = []
m_max_sic_lst = []
m_ortmin_sic_lst = []
m_min_sic_lst = []
m_max_yag_lst = []
m_top_yag_lst = []

for i in range(len(aks_df["ort_sic"])):
    m_ort_sic = (bey_df["ort_sic"][i]+cum_df["ort_sic"][i]+sey_df["ort_sic"][i]+krp_df["ort_sic"][i]+\
                 ere_df["ort_sic"][i]+kon_df["ort_sic"][i]+kul_df["ort_sic"][i]) / 7
    m_ort_sic_lst.append(m_ort_sic)
    
    m_ortmax_sic = (bey_df["ortmax_sic"][i]+cum_df["ortmax_sic"][i]+sey_df["ortmax_sic"][i]+krp_df["ortmax_sic"][i]+\
                 ere_df["ortmax_sic"][i]+kon_df["ortmax_sic"][i]+kul_df["ortmax_sic"][i]) / 7
    m_ortmax_sic_lst.append(m_ortmax_sic)
    
    m_ortmin_sic = (bey_df["ortmin_sic"][i]+cum_df["ortmin_sic"][i]+sey_df["ortmin_sic"][i]+krp_df["ortmin_sic"][i]+\
                 ere_df["ortmin_sic"][i]+kon_df["ortmin_sic"][i]+kul_df["ortmin_sic"][i]) / 7
    m_ortmin_sic_lst.append(m_ortmin_sic)
    
    m_max_sic = (bey_df["max_sic"][i]+cum_df["max_sic"][i]+sey_df["max_sic"][i]+krp_df["max_sic"][i]+\
                 ere_df["max_sic"][i]+kon_df["max_sic"][i]+kul_df["max_sic"][i]) / 7
    m_max_sic_lst.append(m_max_sic)
    
    m_min_sic = (bey_df["min_sic"][i]+cum_df["min_sic"][i]+sey_df["min_sic"][i]+krp_df["min_sic"][i]+\
                 ere_df["min_sic"][i]+kon_df["min_sic"][i]+kul_df["min_sic"][i]) / 7
    m_min_sic_lst.append(m_min_sic)
    
    m_max_yag = (bey_df["max_yag"][i]+cum_df["max_yag"][i]+sey_df["max_yag"][i]+krp_df["max_yag"][i]+\
                 ere_df["max_yag"][i]+kon_df["max_yag"][i]+kul_df["max_yag"][i]) / 7
    m_max_yag_lst.append(m_max_yag)
    
    m_top_yag = (bey_df["top_yag"][i]+cum_df["top_yag"][i]+sey_df["top_yag"][i]+krp_df["top_yag"][i]+\
                 ere_df["top_yag"][i]+kon_df["top_yag"][i]+kul_df["top_yag"][i]) / 7
    m_top_yag_lst.append(m_top_yag)
    
konort_df = pd.DataFrame({"istasyon":"KONYA_ORT",  "Yıl":[x for x in range(2000,2022) for y in range(1,13)], 
                               "Ay":[y for x in range(2000,2022) for y in range(1,13)], 
                          "ort_sic":m_ort_sic_lst, 
                          "ortmax_sic":m_ortmax_sic_lst, "ortmin_sic":m_ortmin_sic_lst, "max_sic":m_max_sic_lst, 
                          "min_sic":m_min_sic_lst, "max_yag":m_max_yag_lst, "top_yag":m_top_yag_lst})
konort_df

,istasyon,Yıl,Ay,ort_sic,ortmax_sic,ortmin_sic,max_sic,min_sic,max_yag,top_yag
0,KONYA_ORT,2000,1,-4.557143,-3.172066e-17,-3.172066e-17,13.771429,-18.271429,NaN,NaN
1,KONYA_ORT,2000,2,-1.842857,2.857143e+00,2.857143e+00,10.500000,-13.328571,NaN,NaN
2,KONYA_ORT,2000,3,3.614286,1.090000e+01,1.090000e+01,23.057143,-12.014286,NaN,NaN
3,KONYA_ORT,2000,4,12.528571,1.948571e+01,1.948571e+01,25.642857,-2.757143,NaN,NaN
4,KONYA_ORT,2000,5,14.185714,2.101429e+01,2.101429e+01,26.328571,2.557143,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
259,KONYA_ORT,2021,8,23.642857,3.135714e+01,3.135714e+01,36.500000,11.485714,4.685714,5.514286
260,KONYA_ORT,2021,9,17.700000,2.484286e+01,2.484286e+01,32.542857,3.514286,12.228571,23.942857
261,KONYA_ORT,2021,10,12.314286,2.011429e+01,2.011429e+01,26.014286,0.514286,0.885714,1.028571
262,KONYA_ORT,2021,11,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [58]:
konort_df.isnull().sum()

istasyon        0
Yıl             0
Ay              0
ort_sic         2
ortmax_sic      3
ortmin_sic      3
max_sic         3
min_sic         3
max_yag       118
top_yag       118
dtype: int64

# Havza NDVI için Hepsinin Ortalamasını Alma

In [59]:
m_ort_sic_lst = []
m_ortmax_sic_lst = []
m_max_sic_lst = []
m_ortmin_sic_lst = []
m_min_sic_lst = []
m_max_yag_lst = []
m_top_yag_lst = []

for i in range(len(aks_df["ort_sic"])):
    m_ort_sic = (bey_df["ort_sic"][i]+cum_df["ort_sic"][i]+sey_df["ort_sic"][i]+krp_df["ort_sic"][i]+\
                 ere_df["ort_sic"][i]+kon_df["ort_sic"][i]+kul_df["ort_sic"][i]+krm_df["ort_sic"][i]+\
                 nig_df["ort_sic"][i]+aks_df["ort_sic"][i]) / 10
    m_ort_sic_lst.append(m_ort_sic)
    
    m_ortmax_sic = (bey_df["ortmax_sic"][i]+cum_df["ortmax_sic"][i]+sey_df["ortmax_sic"][i]+krp_df["ortmax_sic"][i]+\
                 ere_df["ortmax_sic"][i]+kon_df["ortmax_sic"][i]+kul_df["ortmax_sic"][i]+krm_df["ortmax_sic"][i]+\
                 nig_df["ortmax_sic"][i]+aks_df["ortmax_sic"][i]) / 10
    m_ortmax_sic_lst.append(m_ortmax_sic)
    
    m_ortmin_sic = (bey_df["ortmin_sic"][i]+cum_df["ortmin_sic"][i]+sey_df["ortmin_sic"][i]+krp_df["ortmin_sic"][i]+\
                 ere_df["ortmin_sic"][i]+kon_df["ortmin_sic"][i]+kul_df["ortmin_sic"][i]+krm_df["ortmin_sic"][i]+\
                 nig_df["ortmin_sic"][i]+aks_df["ortmin_sic"][i]) / 10
    m_ortmin_sic_lst.append(m_ortmin_sic)
    
    m_max_sic = (bey_df["max_sic"][i]+cum_df["max_sic"][i]+sey_df["max_sic"][i]+krp_df["max_sic"][i]+\
                 ere_df["max_sic"][i]+kon_df["max_sic"][i]+kul_df["max_sic"][i]+krm_df["max_sic"][i]+\
                 nig_df["max_sic"][i]+aks_df["max_sic"][i]) / 10
    m_max_sic_lst.append(m_max_sic)
    
    m_min_sic = (bey_df["min_sic"][i]+cum_df["min_sic"][i]+sey_df["min_sic"][i]+krp_df["min_sic"][i]+\
                 ere_df["min_sic"][i]+kon_df["min_sic"][i]+kul_df["min_sic"][i]+krm_df["min_sic"][i]+\
                 nig_df["min_sic"][i]+aks_df["min_sic"][i]) / 10
    m_min_sic_lst.append(m_min_sic)
    
    m_max_yag = (bey_df["max_yag"][i]+cum_df["max_yag"][i]+sey_df["max_yag"][i]+krp_df["max_yag"][i]+\
                 ere_df["max_yag"][i]+kon_df["max_yag"][i]+kul_df["max_yag"][i]+krm_df["max_yag"][i]+\
                 nig_df["max_yag"][i]+aks_df["max_yag"][i]) / 10
    m_max_yag_lst.append(m_max_yag)
    
    m_top_yag = (bey_df["top_yag"][i]+cum_df["top_yag"][i]+sey_df["top_yag"][i]+krp_df["top_yag"][i]+\
                 ere_df["top_yag"][i]+kon_df["top_yag"][i]+kul_df["top_yag"][i]+krm_df["top_yag"][i]+\
                 nig_df["top_yag"][i]+aks_df["top_yag"][i]) / 10
    m_top_yag_lst.append(m_top_yag)
    
havort_df = pd.DataFrame({"istasyon":"HAVZA_ORT",  "Yıl":[x for x in range(2000,2022) for y in range(1,13)], 
                               "Ay":[y for x in range(2000,2022) for y in range(1,13)],
                          "ort_sic":m_ort_sic_lst, 
                          "ortmax_sic":m_ortmax_sic_lst, "ortmin_sic":m_ortmin_sic_lst, "max_sic":m_max_sic_lst, 
                          "min_sic":m_min_sic_lst, "max_yag":m_max_yag_lst, "top_yag":m_top_yag_lst})
havort_df

,istasyon,Yıl,Ay,ort_sic,ortmax_sic,ortmin_sic,max_sic,min_sic,max_yag,top_yag
0,HAVZA_ORT,2000,1,-4.39,0.22,-2.57,13.80,-18.01,NaN,NaN
1,HAVZA_ORT,2000,2,-1.78,2.92,0.33,10.62,-13.05,NaN,NaN
2,HAVZA_ORT,2000,3,3.67,10.85,6.94,23.29,-11.89,NaN,NaN
3,HAVZA_ORT,2000,4,12.78,19.58,15.87,25.69,-2.42,NaN,NaN
4,HAVZA_ORT,2000,5,14.30,21.03,17.30,26.46,2.99,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
259,HAVZA_ORT,2021,8,23.62,31.19,26.79,36.84,12.01,4.82,6.46
260,HAVZA_ORT,2021,9,17.83,24.89,20.93,32.59,3.86,11.07,20.87
261,HAVZA_ORT,2021,10,12.44,20.23,15.80,26.64,0.55,0.66,0.76
262,HAVZA_ORT,2021,11,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [60]:
havort_df.isnull().sum()

istasyon        0
Yıl             0
Ay              0
ort_sic         3
ortmax_sic      4
ortmin_sic      4
max_sic         4
min_sic         4
max_yag       119
top_yag       119
dtype: int64

In [61]:
"""nanreplace(havort_df, kon_df)"""

'nanreplace(havort_df, kon_df)'

In [62]:
havort_df.isnull().sum()

istasyon        0
Yıl             0
Ay              0
ort_sic         3
ortmax_sic      4
ortmin_sic      4
max_sic         4
min_sic         4
max_yag       119
top_yag       119
dtype: int64

# Nan Değerlerin Silinmesi

Regresyon ve yapay sinir ağlarıyla yaptığımızda etkili bir sonuç elde edemedik. Bu yüzden 2005 Kasım'dan önceki verileri silmemiz gerekmekte.

In [63]:
"""aks_df = aks_df.dropna()
bey_df = bey_df.dropna()
cum_df = cum_df.dropna()
sey_df = sey_df.dropna()
krp_df = krp_df.dropna()
krm_df = krm_df.dropna()
nig_df = nig_df.dropna()
kul_df = kul_df.dropna()
ere_df = ere_df.dropna()
kon_df = kon_df.dropna()
konort_df = konort_df.dropna()
havort_df = havort_df.dropna()"""

'aks_df = aks_df.dropna()\nbey_df = bey_df.dropna()\ncum_df = cum_df.dropna()\nsey_df = sey_df.dropna()\nkrp_df = krp_df.dropna()\nkrm_df = krm_df.dropna()\nnig_df = nig_df.dropna()\nkul_df = kul_df.dropna()\nere_df = ere_df.dropna()\nkon_df = kon_df.dropna()\nkonort_df = konort_df.dropna()\nhavort_df = havort_df.dropna()'

In [64]:
krm_df

,İstasyon,Yıl,Ay,max_sic,max_yag,min_sic,ortmin_sic,ort_sic,top_yag,ortmax_sic
0,Karaman,2000,1,13.5,NaN,-18.2,-8.6,-3.8,NaN,1.0
1,Karaman,2000,2,11.3,NaN,-10.4,-4.6,-0.7,NaN,3.9
2,Karaman,2000,3,26.4,NaN,-14.8,-2.7,4.2,NaN,11.6
3,Karaman,2000,4,25.4,NaN,-3.2,6.9,13.4,NaN,20.2
4,Karaman,2000,5,27.6,NaN,3.7,8.2,14.6,NaN,21.7
...,...,...,...,...,...,...,...,...,...,...
259,Karaman,2021,8,37.2,0.0,12.3,15.8,23.6,0.0,30.9
260,Karaman,2021,9,33.2,4.4,4.2,11.7,18.4,7.6,25.4
261,Karaman,2021,10,29.7,0.0,0.7,5.4,13.0,0.0,21.0
262,Karaman,2021,11,22.7,6.4,-4.3,3.7,9.3,14.4,15.9


In [65]:
aks_df

,İstasyon,Yıl,Ay,max_sic,max_yag,min_sic,ortmin_sic,ort_sic,top_yag,ortmax_sic
0,Aksaray,2000,1,14.2,NaN,-16.0,-8.1,-3.7,NaN,0.6
1,Aksaray,2000,2,11.9,NaN,-12.0,-5.2,-1.5,NaN,3.1
2,Aksaray,2000,3,23.2,NaN,-9.3,-1.4,4.3,NaN,10.9
3,Aksaray,2000,4,26.6,NaN,-2.0,7.8,13.8,NaN,20.4
4,Aksaray,2000,5,27.0,NaN,4.7,9.6,15.2,NaN,21.0
...,...,...,...,...,...,...,...,...,...,...
259,Aksaray,2021,8,37.6,7.6,14.6,17.1,24.3,16.4,31.3
260,Aksaray,2021,9,33.3,9.3,5.8,12.7,18.4,12.7,24.9
261,Aksaray,2021,10,27.2,0.0,1.8,6.7,13.0,0.0,20.5
262,Aksaray,2021,11,22.0,9.4,-5.1,4.4,9.7,17.7,16.5


In [66]:
havort_df

,istasyon,Yıl,Ay,ort_sic,ortmax_sic,ortmin_sic,max_sic,min_sic,max_yag,top_yag
0,HAVZA_ORT,2000,1,-4.39,0.22,-2.57,13.80,-18.01,NaN,NaN
1,HAVZA_ORT,2000,2,-1.78,2.92,0.33,10.62,-13.05,NaN,NaN
2,HAVZA_ORT,2000,3,3.67,10.85,6.94,23.29,-11.89,NaN,NaN
3,HAVZA_ORT,2000,4,12.78,19.58,15.87,25.69,-2.42,NaN,NaN
4,HAVZA_ORT,2000,5,14.30,21.03,17.30,26.46,2.99,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
259,HAVZA_ORT,2021,8,23.62,31.19,26.79,36.84,12.01,4.82,6.46
260,HAVZA_ORT,2021,9,17.83,24.89,20.93,32.59,3.86,11.07,20.87
261,HAVZA_ORT,2021,10,12.44,20.23,15.80,26.64,0.55,0.66,0.76
262,HAVZA_ORT,2021,11,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Sonra kullanmak için kaydedelim.

In [67]:
aks_df.to_csv("Aksaray_Istasyon.csv")
bey_df.to_csv("Beysehir_Istasyon.csv")
cum_df.to_csv("Cumra_Istasyon.csv")
sey_df.to_csv("Seydisehir_Istasyon.csv")
krp_df.to_csv("Karapinar_Istasyon.csv")
krm_df.to_csv("Karaman_Istasyon.csv")
nig_df.to_csv("Nigde_Istasyon.csv")
kul_df.to_csv("Kulu_Istasyon.csv")
ere_df.to_csv("Eregli_Istasyon.csv")
kon_df.to_csv("Konya_Istasyon.csv")
konort_df.to_csv("KonOrt_Istasyon.csv")
havort_df.to_csv("HavOrt_Istasyon.csv")